In [1]:
x=1

In [2]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import decoupler as dc
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad
import random
import sc_toolbox
import pertpy 

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

To use sccoda or tasccoda please install ete3 with pip install ete3

In [3]:
adata_embed = sc.read('../../../../pipeline/data/multigrate_mil_reg/pbmc_healthy_covid_3cond_mtg_embed_ord_reg/959918ced7_adata_both.h5ad')
adata_embed

AnnData object with n_obs × n_vars = 362426 × 16
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'split0', 'split1', 'split2', 'split3', 'split4', 'group', 'size_factors', '_scvi_batch', 'Status_on_day_collection_summary_cont', 'cell_attn_0', 'cell_attn_1', 'cell_attn_2', 'cell_attn_3', 'cell_attn_4', 'cell_attn'
    obsm: 'X_umap', 'latent_0', 'latent_1', 'latent_2', 'latent_3', 'latent_4'

In [4]:
adata_counts = sc.read('../../../../pipeline/data/pp/mtg3_clean_healthy_covid_embed.h5ad')
adata_counts

AnnData object with n_obs × n_vars = 362426 × 2192
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'split0', 'split1', 'split2', 'split3', 'split4', 'group', 'size_factors', '_scvi_batch'
    var: 'modality'
    uns: 'Site_colors', 'Status_on_day_collection_summary_colors', '_scvi_manager_uuid', '_scvi_uuid', 'full_clustering_colors', 'initial_clustering_colors', 'modality_lengths', 'neighbors', 'patient_id_colors', 'umap'
    obsm: 'X_umap', '_scvi_extra_categorical_covs', 'latent'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [5]:
adata = sc.AnnData(adata_counts.layers['counts'][:, :2000])
adata.obs_names = adata_counts.obs_names
adata.var_names = adata_counts.var_names[:2000]
adata.obs = adata.obs.join(adata_embed.obs)
adata.obsm = adata_counts.obsm
adata

AnnData object with n_obs × n_vars = 362426 × 2000
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'split0', 'split1', 'split2', 'split3', 'split4', 'group', 'size_factors', '_scvi_batch', 'Status_on_day_collection_summary_cont', 'cell_attn_0', 'cell_attn_1', 'cell_attn_2', 'cell_attn_3', 'cell_attn_4', 'cell_attn'
    obsm: 'X_umap', '_scvi_extra_categorical_covs', 'latent'

In [6]:
adata.X.data

array([15.,  1., 30., ...,  7.,  3.,  7.], dtype=float32)

In [7]:
import rpy2.robjects as ro

def run_de(adata):
    r=ro.r
    getwd = ro.r['getwd']
    print(getwd())
    r.source('../r_scripts/run_de.R')
    df = r.fit_model(adata)
    return df

In [8]:
adata.obs['Status_on_day_collection_summary']

covid_index
AAACCTGAGACCACGA-newcastle65    Healthy
AAACCTGAGATGTCGG-newcastle65    Healthy
AAACCTGAGGCGATAC-newcastle65    Healthy
AAACCTGAGTACACCT-newcastle65    Healthy
AAACCTGAGTGAATTG-newcastle65    Healthy
                                 ...   
BGCV15_TTTGTCAAGGCGATAC-1        Severe
BGCV15_TTTGTCACAAGCCATT-1       Healthy
BGCV15_TTTGTCACAGACACTT-1          Mild
BGCV15_TTTGTCAGTTACGGAG-1          Mild
BGCV15_TTTGTCATCGAATGGG-1        Severe
Name: Status_on_day_collection_summary, Length: 362426, dtype: category
Categories (3, object): ['Healthy', 'Mild', 'Severe']

In [9]:
adata.obs['initial_clustering']

covid_index
AAACCTGAGACCACGA-newcastle65        CD4
AAACCTGAGATGTCGG-newcastle65    NK_16hi
AAACCTGAGGCGATAC-newcastle65       Treg
AAACCTGAGTACACCT-newcastle65    NK_16hi
AAACCTGAGTGAATTG-newcastle65        CD4
                                 ...   
BGCV15_TTTGTCAAGGCGATAC-1           CD4
BGCV15_TTTGTCACAAGCCATT-1       NK_16hi
BGCV15_TTTGTCACAGACACTT-1           CD8
BGCV15_TTTGTCAGTTACGGAG-1           CD4
BGCV15_TTTGTCATCGAATGGG-1       NK_16hi
Name: initial_clustering, Length: 362426, dtype: category
Categories (18, object): ['B_cell', 'CD4', 'CD8', 'CD14', ..., 'RBC', 'Treg', 'gdT', 'pDC']

test top in mild vs top in severe

In [10]:
adata.obs['celltype'] = adata.obs['initial_clustering']
adata.obs['condition'] = adata.obs['Status_on_day_collection_summary']
adata.obs['sample'] = adata.obs['patient_id']
adata.obs['study'] = adata.obs['Site']

In [11]:
celltype = 'celltype'
condition_col = 'condition'
condA = 'Mild'
condB = 'Severe'
sample = 'sample'
study = 'study'

In [12]:
adata.layers['counts'] = adata.X.copy()

In [14]:
for ct in np.unique(adata.obs['initial_clustering'].astype(str)):
    print(f'Processing {ct}...')
    adata_ct = adata[adata.obs[celltype] == ct].copy()
    adata_ct = adata_ct[adata_ct.obs[condition_col].isin([condA, condB])].copy()

    # print(len(adata_ct_disease))

    adata_condA = adata_ct[adata_ct.obs[condition_col].isin([condA])].copy()
    adata_condB = adata_ct[adata_ct.obs[condition_col].isin([condB])].copy()

    for adata_tmp in [adata_condA, adata_condB]:
        threshold_idx = int(len(adata_tmp) * 0.9)
        threshold_value = sorted(adata_tmp.obs['cell_attn'])[threshold_idx]
        
        adata_tmp.obs['group'] = 'all'
        adata_tmp.obs.loc[adata_tmp.obs['cell_attn'] >= threshold_value, 'group'] = 'top'

    adata_ct = ad.concat([adata_condA, adata_condB])
    # subset to otp cells only
    adata_ct = adata_ct[adata_ct.obs['group'] == 'top'].copy()
    
    # adata_ct_disease.obs['sample_group'] = adata_ct_disease.obs['sample'].astype('str') + '_' + adata_ct_disease.obs['group']
    # adata_ct_disease.obs['sample_group'] = adata_ct_disease.obs['sample_group'].astype('category')

    # samples_to_exclude = []
    # for sample in np.unique(adata_ct_disease.obs[['sample_group']]):
    #     sample_len = len(adata_ct_disease[adata_ct_disease.obs['sample_group'] == sample])
    #     if sample_len < 10:
    #         samples_to_exclude.append(sample)
    
    # adata_ct_disease = adata_ct_disease[~adata_ct_disease.obs['sample_group'].isin(samples_to_exclude)].copy()

    #adata_ct_disease.X = adata_ct_disease.obsm['counts'].copy()
    adata_pb = sc.AnnData(adata_ct.layers['counts'])
    adata_pb.obs = adata_ct.obs
    #print(adata_pb)
    adata_pb.var_names = adata_ct.var_names

    adata_ = dc.get_pseudobulk(adata_pb, sample, condition_col, mode='sum', min_cells=-1, min_counts=10)
    #print(adata_)

    adata_r = sc.AnnData(adata_.X, obs=adata_.obs[[condition_col, study, sample]])
    adata_r.var_names = adata_.var_names
    adata_r.obs[sample] = adata_r.obs[sample].str.replace('-', '_')
    adata_r.obs.index = adata_r.obs.index.str.replace('-', '_')

    adata_r = adata_r.copy()

    sc.pp.filter_genes(adata_r, min_cells=10)

    if ((adata_r.X.shape[0] == 0) or (adata_r.X.shape[1] == 0)):
        continue
    df = run_de(adata_r)

    df = ro.conversion.rpy2py(df)
    df.to_csv(f'de_results/top_mild_vs_top_severe_{ct}.csv')

Processing B_cell...
[1] "/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/multimil_reproducibility/notebooks/mil_analysis/pbmc/ord_regression"

[1] "Dimensions before subsetting:"
[1] 893  44
[1] ""
[1] "Dimensions after subsetting:"
[1] 325  44
[1] ""
   conditionMild conditionSevere studyNcl studySanger
1              1               0        0           1
2              1               0        0           1
3              1               0        0           0
4              1               0        0           0
5              1               0        0           0
6              1               0        0           0
7              1               0        0           0
8              1               0        0           0
9              1               0        0           0
10             1               0        0           0
11             1               0        0           0
12             1               0        1           0
13             1         